# Realized moments estimations

In [1]:
# export
import numpy as np

In [3]:
import yfinance as yf
import pandas as pd
assets = ['AAPL', 'MSFT']
start='2017-12-01'
end='2018-01-01'
prices = yf.download(assets, start = start, end = end)
prices.index = pd.to_datetime(prices.index, utc=True)
prices.head()

[*********************100%***********************]  2 of 2 completed


Adj Close                 Close             \
                                AAPL       MSFT       AAPL       MSFT   
Date                                                                    
2017-12-01 05:00:00+00:00  40.544769  78.977997  42.762501  84.260002   
2017-12-04 05:00:00+00:00  40.248489  75.997337  42.450001  81.080002   
2017-12-05 05:00:00+00:00  40.210564  76.475357  42.410000  81.589996   
2017-12-06 05:00:00+00:00  40.061234  77.590759  42.252499  82.779999   
2017-12-07 05:00:00+00:00  40.134708  77.318954  42.330002  82.489998   

                                High                   Low             \
                                AAPL       MSFT       AAPL       MSFT   
Date                                                                    
2017-12-01 05:00:00+00:00  42.917500  84.809998  42.125000  83.220001   
2017-12-04 05:00:00+00:00  43.154999  84.430000  42.407501  80.699997   
2017-12-05 05:00:00+00:00  42.880001  82.680000  42.099998  80.980003   
2017-12-06 05:00:00+00:00  42.549999  83.139999  41.615002  81.430000   
2017-12-07 05:00:00+00:00  42.610001  82.800003  42.227501  82.000000   

                                Open                Volume            
                                AAPL       MSFT       AAPL      MSFT  
Date                                                                  
2017-12-01 05:00:00+00:00  42.487499  83.599998  159037200  29532100  
2017-12-04 05:00:00+00:00  43.119999  84.419998  130169600  39094900  
2017-12-05 05:00:00+00:00  42.264999  81.339996  109400800  26152300  
2017-12-06 05:00:00+00:00  41.875000  81.550003  114240000  26162100  
2017-12-07 05:00:00+00:00  42.257500  82.540001  102693200  23184500

In [10]:
ret=np.log(prices['Adj Close'].astype('float')).diff().dropna()
ret.head()

,AAPL,MSFT
Date,,
2017-12-04 05:00:00+00:00,-0.007334,-0.038471
2017-12-05 05:00:00+00:00,-0.000943,0.006270
2017-12-06 05:00:00+00:00,-0.003721,0.014480
2017-12-07 05:00:00+00:00,0.001832,-0.003509
2017-12-08 05:00:00+00:00,0.000295,0.020043


## Realized variance
$R D V a r_{t}=\sum_{i=1}^{N}r_{t,i}^{2}$

In [2]:
# export
def realized_var(arr):
    return np.sum(np.power(arr, 2))

In [11]:
ret.resample('W').apply(realized_var)

,AAPL,MSFT
Date,,
2017-12-10 00:00:00+00:00,0.000072,0.002143
2017-12-17 00:00:00+00:00,0.000517,0.000878
2017-12-24 00:00:00+00:00,0.000326,0.000083
2017-12-31 00:00:00+00:00,0.000786,0.000019


## Realized skew
$R D S k e w_{t}=\frac{\sqrt{\mathcal{N}}\sum_{i=1}^{N}\,r_{t,i}^{3}}{R D V_{a r_{t}}{}^{3/2}}$

In [13]:
# export
def realized_skew(arr):
    if arr.shape[0] == 0:
        # corner case
        return np.nan
    N = arr.shape[0] ** 0.5
    var = realized_var(arr)
    moment = np.sum(np.power(arr, 3))
    # Naive: 
    return N*moment/np.power(var,1.5)
    # Move to log solution
    # logSkew=np.log(N)
    # try:
    #     logSkew=logSkew + np.log(moment)
    # except:
    #     print(moment)
    #     raise
    # logSkew = logSkew - 1.5*np.log(var)
    # return np.exp(logSkew)

In [14]:
ret.resample('W').apply(realized_skew)

,AAPL,MSFT
Date,,
2017-12-10 00:00:00+00:00,-1.614007,-1.028861
2017-12-17 00:00:00+00:00,1.534556,1.509853
2017-12-24 00:00:00+00:00,0.592345,-1.373175
2017-12-31 00:00:00+00:00,-1.653241,0.858185


## Realized kurtosis
$R D K u r t_{t}=\frac{N\sum_{i=1}^{N}r_{t,i}^{4}}{R D Var_{t}^{2}}$

In [15]:
# export
def realized_kurt(arr):
    if arr.shape[0] == 0:
        # corner case
        return np.nan
    N = arr.shape[0]
    var = realized_var(arr)
    moment = np.sum(np.power(arr, 4))
    return N*moment/np.power(var,2)

In [16]:
ret.resample('W').apply(realized_kurt)

,AAPL,MSFT
Date,,
2017-12-10 00:00:00+00:00,2.989961,2.610180
2017-12-17 00:00:00+00:00,2.806083,2.799173
2017-12-24 00:00:00+00:00,2.431731,1.942851
2017-12-31 00:00:00+00:00,2.910645,2.107933


In [17]:
from python_aux import notebook2script
notebook2script('realized_moments.ipynb')

Converted realized_moments.ipynb to ../src/realized_moments.py
